# How can the Zeus Subnet be used to query miners on live data?

We aim to make it as easy as possible to build a product around this subnet. By default, a validator will automatically start a Proxy process, which allows it to query miners using data of the validator's choice! This is all seamlessly integrated with the main validator logic, meaning a miner will not know whether an incoming request concerns one they will be scored on, or one that is used for a downstream product.

This notebook will briefly demonstrate how to use the Proxy API. Note that it is very simple to modify it for your own use-case, which can be done by modifying $\texttt{zeus->api->proxy.py}$.
All proxy options can be configured in the validator.env file

In [ ]:
import requests
import torch
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib
import matplotlib.pyplot as plt

from datetime import datetime, timedelta

### Request the API
It is of course possible to change the latitude/longitude bounding box, the number of hours the miners are shown, and the number of hours you want them to predict.
Going outside the ranges configured for a validator is possible, but will allow miners to distinguish (and potentially filter out) a Proxy and regular (scored) request.

In [ ]:
# get the timestamp of yesterday 1 o clock
yesterday1o = datetime.now().replace(hour=1, minute=0, second=0, microsecond=0) - timedelta(days=1)
yesterday3o = datetime.now().replace(hour=1, minute=0, second=0, microsecond=0)

response = requests.post(
    url='http://localhost:<PROXY_PORT>/predictPoint',
    headers={
        'authorization': '<PROXY_API_KEY>',
    },
    json={
        "lat":51,  # coordinates of the Netherlands
        "lon": 54,
        "start_timestamp": float(yesterday1o.timestamp()),
        "end_timestamp": float(yesterday3o.timestamp()),
        'variable': "2m_temperature",
    }
)

In [ ]:
if response.status_code == 200:
        prediction = torch.tensor(response.json()["prediction"])
        print(prediction.shape)